1.导入必要的库

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

2.数据读取与整合

2.1 读取门禁数据

In [4]:
# 门禁数据路径
entry_files = [
    "mj20140901-20150831.txt", 
    "mj20150901-20160831.txt", 
    "mj20160901-20170831.txt", 
    "mj20170901-20180831.txt"
]

entry_data = []

In [5]:
# 遍历每个文件读取数据
for file in entry_files:
    df = pd.read_csv(file, sep="\t", names=["ID", "VisitTime", "Location"], encoding="utf-8", header=None)
    entry_data.append(df)

# 合并所有门禁数据
entry_logs = pd.concat(entry_data, ignore_index=True)

# 检查空值
print("是否存在空值：", entry_logs['VisitTime'].isnull().sum())

# 检查非时间格式值
invalid_times = entry_logs[~entry_logs['VisitTime'].str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}$')]
print("非时间格式的值：")
print(invalid_times)

# 解析时间格式
entry_logs['VisitTime'] = pd.to_datetime(entry_logs['VisitTime'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

# 删除解析失败的行
entry_logs = entry_logs.dropna(subset=['VisitTime'])

# 输出解析后的数据
print("门禁数据预览：")
print(entry_logs.head())


是否存在空值： 0
非时间格式的值：
         ID  VisitTime  Location
0        ID  VisitTime  Location
1968250  ID  VisitTime  Location
4119015  ID  VisitTime  Location
6415228  ID  VisitTime  Location
门禁数据预览：
                                 ID               VisitTime Location
1  00AAF5172011E421E06324DC10AC52C4 2015-03-30 14:33:28.307     思明校区
2  00AAF5171D07E421E06324DC10AC52C4 2015-03-30 14:34:55.807     思明校区
3  00AAF5170243E421E06324DC10AC52C4 2015-03-30 14:36:29.240     思明校区
4  00AAF516E804E421E06324DC10AC52C4 2015-03-30 14:37:36.797     思明校区
5  00AAF5170D6DE421E06324DC10AC52C4 2015-03-30 14:38:31.170     思明校区


2.2 读取借阅数据

In [8]:
import pandas as pd
import chardet
from datetime import datetime

# 定义一个函数将文件统一转换为 UTF-8 编码
def convert_to_utf8(input_file, output_file):
    """
    将文件编码转换为 UTF-8
    :param input_file: 输入文件路径
    :param output_file: 输出文件路径（转换为 UTF-8）
    """
    with open(input_file, 'rb') as f:
        raw_data = f.read()
    # 检测文件编码
    detected_encoding = chardet.detect(raw_data)['encoding']
    print(f"文件 {input_file} 检测到编码：{detected_encoding}")
    
    # 读取文件并转换编码
    with open(input_file, 'r', encoding=detected_encoding, errors='replace') as f:
        content = f.read()
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"文件 {input_file} 已转换为 UTF-8 编码并保存为 {output_file}")

# 定义文件路径
files_to_convert = {
    "读者数据_guid.csv": "读者数据_guid_utf8.csv",
    "借阅数据_guid.csv": "借阅数据_guid_utf8.csv",
    "图书数据.csv": "图书数据_utf8.csv"
}

# 转换文件为 UTF-8 编码
for input_file, output_file in files_to_convert.items():
    convert_to_utf8(input_file, output_file)

# 读取转换后的 UTF-8 文件

# 读取读者数据
reader_data = pd.read_csv(files_to_convert["读者数据_guid.csv"], encoding="utf-8")
reader_data.columns = ["NO", "ID", "Gender", "EnrollYear", "Type", "Department"]
reader_data = reader_data.drop(columns=["NO"])

# 读取借阅记录
borrow_logs = pd.read_csv(files_to_convert["借阅数据_guid.csv"], encoding="utf-8")
borrow_logs.columns = ["NO", "ReaderID", "BookID", "LendDate", "ReturnDate", "RenewCounts"]
borrow_logs = borrow_logs.drop(columns=["NO"])

# 转换借阅和归还日期为 datetime 类型
# 字段原格式为字符串，形如 "2014-09-01 10:12:26"
borrow_logs["LendDate"] = pd.to_datetime(borrow_logs["LendDate"], format="%Y-%m-%d %H:%M:%S", errors='coerce')
borrow_logs["ReturnDate"] = pd.to_datetime(borrow_logs["ReturnDate"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# 检查是否有转换失败的值
print("转换失败的日期值（LendDate）：")
print(borrow_logs[borrow_logs["LendDate"].isnull()])

print("转换失败的日期值（ReturnDate）：")
print(borrow_logs[borrow_logs["ReturnDate"].isnull()])

# 删除转换失败的日期值（可选）
borrow_logs = borrow_logs.dropna(subset=["LendDate", "ReturnDate"])

# 读取图书数据
book_data = pd.read_csv(files_to_convert["图书数据.csv"], encoding="utf-8")
book_data.columns = ["NO", "ID", "Title", "Author", "Publisher", "Year", "CallNo", "Language", "DocType"]
book_data = book_data.drop(columns=["NO"])

# 查看数据预览
print("读者数据预览：")
print(reader_data.head())

print("\n借阅记录数据预览：")
print(borrow_logs.head())

print("\n图书数据预览：")
print(book_data.head())

文件 读者数据_guid.csv 检测到编码：GB2312
文件 读者数据_guid.csv 已转换为 UTF-8 编码并保存为 读者数据_guid_utf8.csv
文件 借阅数据_guid.csv 检测到编码：ascii
文件 借阅数据_guid.csv 已转换为 UTF-8 编码并保存为 借阅数据_guid_utf8.csv
文件 图书数据.csv 检测到编码：UTF-8-SIG
文件 图书数据.csv 已转换为 UTF-8 编码并保存为 图书数据_utf8.csv
转换失败的日期值（LendDate）：
Empty DataFrame
Columns: [ReaderID, BookID, LendDate, ReturnDate, RenewCounts]
Index: []
转换失败的日期值（ReturnDate）：
Empty DataFrame
Columns: [ReaderID, BookID, LendDate, ReturnDate, RenewCounts]
Index: []
读者数据预览：
                                 ID Gender EnrollYear Type Department
0  00AAF5172AB1E421E06324DC10AC52C4      M       2013  本科生        药学院
1  00AAF516FA58E421E06324DC10AC52C4      M       2012  研究生       经济学院
2  00AAF5170950E421E06324DC10AC52C4      M       2012  研究生      台湾研究院
3  00AAF516D41BE421E06324DC10AC52C4      F       2011  研究生     航空航天学院
4  00AAF516FCD4E421E06324DC10AC52C4      M       2012  研究生       经济学院

借阅记录数据预览：
                           ReaderID                            BookID  \
0  00AAF51729B0E421E06324DC10

2.3 读取座位使用数据

In [10]:
# 座位使用记录文件路径
seat_files = [
    "20140901-20150831.txt",
    "20150901-20160831.txt",
    "20160901-20170831.txt",
    "20170901-20180831.txt"
]

seat_data = []

In [12]:
# 遍历每个文件读取数据
for file in seat_files:
    # 读取座位使用数据
    df = pd.read_csv(file, sep="\t", names=["ID", "ReadingRoomNo", "SeatNo", "SelectSeatTime", "LeaveSeatTime"], encoding="utf-8", header=None)
    seat_data.append(df)

# 合并所有座位数据
seat_logs = pd.concat(seat_data, ignore_index=True)

# 检查空值
print("是否存在空值：")
print(seat_logs.isnull().sum())

# 检查非时间格式值（SelectSeatTime 和 LeaveSeatTime）
invalid_select_times = seat_logs[~seat_logs['SelectSeatTime'].str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}$', na=False)]
invalid_leave_times = seat_logs[~seat_logs['LeaveSeatTime'].str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}$', na=False)]

print("\n非时间格式的值（SelectSeatTime）：")
print(invalid_select_times)

print("\n非时间格式的值（LeaveSeatTime）：")
print(invalid_leave_times)

# 转换时间字段为 datetime 类型
seat_logs["SelectSeatTime"] = pd.to_datetime(seat_logs["SelectSeatTime"], format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")
seat_logs["LeaveSeatTime"] = pd.to_datetime(seat_logs["LeaveSeatTime"], format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")

# 删除解析失败的行
seat_logs = seat_logs.dropna(subset=["SelectSeatTime", "LeaveSeatTime"])

# 输出解析后的数据
print("\n解析后的座位使用数据预览：")
print(seat_logs.head())

C:\Users\Yvett\AppData\Local\Temp\ipykernel_77496\86232910.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t", names=["ID", "ReadingRoomNo", "SeatNo", "SelectSeatTime", "LeaveSeatTime"], encoding="utf-8", header=None)
C:\Users\Yvett\AppData\Local\Temp\ipykernel_77496\86232910.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t", names=["ID", "ReadingRoomNo", "SeatNo", "SelectSeatTime", "LeaveSeatTime"], encoding="utf-8", header=None)
C:\Users\Yvett\AppData\Local\Temp\ipykernel_77496\86232910.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t", names=["ID", "ReadingRoomNo", "SeatNo", "SelectSeatTime", "LeaveSeatTime"], encoding="utf-8", header=None)
C:\Users\Yvett\AppData\Local\Temp\ipykernel_77496\86232910.py:4: DtypeWar

是否存在空值：
ID                0
ReadingRoomNo     0
SeatNo            0
SelectSeatTime    0
LeaveSeatTime     0
dtype: int64

非时间格式的值（SelectSeatTime）：
         ID  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
0        id  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
941873   ID  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
1941114  ID  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
2951769  ID  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
3942001  id  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
4883874  ID  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
5883115  ID  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
6893770  ID  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime

非时间格式的值（LeaveSeatTime）：
         ID  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
0        id  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
941873   ID  ReadingRoomNo  SeatNo  SelectSeatTime  LeaveSeatTime
1941114  ID  ReadingRoomNo  SeatNo  

2.4 整合所有数据

In [17]:
import os

# 定义保存路径
folder_path = "C:/Users/Yvett/PycharmProjects/pythonProject/图书馆"
file_path = os.path.join(folder_path, "entry_logs.csv")

# 检查文件夹是否存在，如果不存在则创建
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"创建文件夹：{folder_path}")

# 将 entry_logs 写入 CSV 文件
entry_logs.to_csv(file_path, index=False, encoding="utf-8")

print(f"entry_logs 已成功保存为 CSV 文件：{file_path}")

entry_logs 已成功保存为 CSV 文件：C:/Users/Yvett/PycharmProjects/pythonProject/图书馆\entry_logs.csv


In [18]:
# 定义保存路径
folder_path = "C:/Users/Yvett/PycharmProjects/pythonProject/图书馆"
file_path = os.path.join(folder_path, "seat_logs.csv")

# 检查文件夹是否存在，如果不存在则创建
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"创建文件夹：{folder_path}")

# 将 seat_logs 写入 CSV 文件
seat_logs.to_csv(file_path, index=False, encoding="utf-8")

print(f"seat_logs 已成功保存为 CSV 文件：{file_path}")

seat_logs 已成功保存为 CSV 文件：C:/Users/Yvett/PycharmProjects/pythonProject/图书馆\seat_logs.csv
